# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pp

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

file = '../WeatherPy/output_data/cities_weather.csv'
data = pd.read_csv(file)

df = pd.DataFrame(data)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,63.54,100,75,4.61,ZA,1666410038
1,Blackwater,-23.5833,148.8833,84.40,42,6,7.70,AU,1666410039
2,Hithadhoo,-0.6000,73.0833,81.23,75,52,7.02,MV,1666410039
3,Butaritari,3.0707,172.7902,81.72,69,8,10.74,KI,1666410039
4,Ridgecrest,35.6225,-117.6709,76.64,20,0,9.22,US,1666409906


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [101]:
# Access maps with API key
gmaps.configure(api_key=g_key)

In [108]:
# create list of locations by lat/long and weights by humidity
places = [(df.loc[i,'Lat'],df.loc[i,'Lng']) for i in df.index]
humidity = [df.loc[i,'Humidity'] for i in df.index]

figure_layout = {
    'width': '700px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(map_type='HYBRID',layout=figure_layout)
heat_layer = gmaps.heatmap_layer(places,
                                 dissipating=False,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [109]:
hotel_df = df.loc[
    (df['Max Temp'] < 80) & 
    (df['Max Temp'] > 70) & 
    (df['Wind Speed'] < 10) &
    (df['Cloudiness'] == 0)
]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Ridgecrest,35.6225,-117.6709,76.64,20,0,9.22,US,1666409906
48,Saint George,37.1041,-113.5841,70.39,33,0,1.01,US,1666410051
139,Salalah,17.0151,54.0924,75.29,78,0,3.44,OM,1666410079
226,Saint-Pierre,-21.3393,55.4781,73.90,60,0,3.44,RE,1666410105
353,Wāliṅ,27.9833,83.7667,72.88,40,0,1.32,NP,1666410142
360,Beloha,-25.1667,45.0500,74.64,70,0,6.55,MG,1666410144
515,Madera,37.1666,-119.8343,72.19,28,0,4.68,US,1666410169
543,Dārāb,28.7519,54.5444,72.37,19,0,1.63,IR,1666410196


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [110]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    'key':g_key,
    'radius': 50000,
    'types': 'lodging',
}

for i in hotel_df.index:
    lat = hotel_df.loc[i,'Lat']
    lng = hotel_df.loc[i,'Lng']
    
    params['location'] = f'{lat},{lng}'
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try: 
        first_result = results[0]
        pp.pprint(first_result)
        print()
    except(IndexError):
        print('No results found')
    
    hotel_df.loc[i, 'Hotel Name'] = first_result.get('name')

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 35.64064369999999, 'lng': -117.6679547},
              'viewport': {'northeast': {'lat': 35.6419597302915,
                                         'lng': -117.6661654},
                           'southwest': {'lat': 35.6392617697085,
                                         'lng': -117.669329}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'icon_background_color': '#909CE1',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
 'name': 'SpringHill Suites by Marriott Ridgecrest',
 'photos': [{'height': 320,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/100181840956667329362">SpringHill '
                                   'Suites by Marriott Ridgecrest</a>'],
             'photo_reference': 'AcYSjRiVR8c_xEGAGCGVdtA2wm7qKPOBHdYB9aqzzq7OW0EnDfsqCYiZ-fruBUWDTLxUhbqm

c:\Users\parkm\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\parkm\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 37.0890245, 'lng': -113.5842669},
              'viewport': {'northeast': {'lat': 37.0901987302915,
                                         'lng': -113.5824545},
                           'southwest': {'lat': 37.08750076970851,
                                         'lng': -113.5864653}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'icon_background_color': '#909CE1',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
 'name': 'Best Western Plus Abbey Inn',
 'photos': [{'height': 2160,
             'html_attributions': ['<a '
                                   'href="https://maps.google.com/maps/contrib/110580949539846287401">Best '
                                   'Western Plus Abbey Inn</a>'],
             'photo_reference': 'AcYSjRg8u5sNJ-t22cdv_kilns4GUYgw1lIO0wxjqFxQJ2Sj6tJ_bgWXhB8UWmkRbL3_qC25bOWYBblMGDktVpj3SNz5IB7lMpxNB9

In [105]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Ridgecrest,35.6225,-117.6709,76.64,20,0,9.22,US,1666409906,SpringHill Suites by Marriott Ridgecrest
48,Saint George,37.1041,-113.5841,70.39,33,0,1.01,US,1666410051,Best Western Plus Abbey Inn
139,Salalah,17.0151,54.0924,75.29,78,0,3.44,OM,1666410079,Hilton Salalah Resort
226,Saint-Pierre,-21.3393,55.4781,73.90,60,0,3.44,RE,1666410105,Alize Plage
353,Wāliṅ,27.9833,83.7667,72.88,40,0,1.32,NP,1666410142,Dangse
360,Beloha,-25.1667,45.0500,74.64,70,0,6.55,MG,1666410144,Marokala Ouest
515,Madera,37.1666,-119.8343,72.19,28,0,4.68,US,1666410169,Chukchansi Gold Resort & Casino
543,Dārāb,28.7519,54.5444,72.37,19,0,1.63,IR,1666410196,Naghshe shapoor Hotel


In [111]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [113]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='700px'))